<a href="https://colab.research.google.com/github/Elllifa/Binary_image_classification/blob/master/Cleaned_Dirty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications, optimizers

Using TensorFlow backend.


In [0]:
img_size = 224
batch_size = 32

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import zipfile
fantasy_zip = zipfile.ZipFile('../content/drive/My Drive/plates.zip')
fantasy_zip.extractall('../content/')

In [5]:
train_datagen=ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True
        )

train_generator = train_datagen.flow_from_directory(
        '../content/plates/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary')

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(  
        '../content/plates',
        classes=['test'],
        target_size = (img_size, img_size),
        batch_size = 1,
        shuffle = False,        
        class_mode = None)


Found 40 images belonging to 2 classes.
Found 744 images belonging to 1 classes.


In [6]:
base_model = applications.InceptionResNetV2(weights='imagenet', 
                          include_top=False, 
                          input_shape=(img_size, img_size, 3))














219062272/219055592 [==============================] - 7s 0us/step


In [0]:
base_model.trainable = False

In [9]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)
model = Model(input = base_model.input, output = predictions)

model.compile(loss='binary_crossentropy', optimizer = optimizers.rmsprop(lr=0.001, decay=1e-6), metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [10]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=25,
        verbose=1)



Epoch 1/25
20/20 [==============================] - 51s 3s/step - loss: 2.4042 - acc: 0.4823
Epoch 2/25
20/20 [==============================] - 9s 448ms/step - loss: 1.1989 - acc: 0.5627
Epoch 3/25
20/20 [==============================] - 9s 426ms/step - loss: 1.2132 - acc: 0.7655
Epoch 4/25
20/20 [==============================] - 9s 431ms/step - loss: 0.7031 - acc: 0.8445
Epoch 5/25
20/20 [==============================] - 8s 421ms/step - loss: 0.8191 - acc: 0.8205
Epoch 6/25
20/20 [==============================] - 9s 427ms/step - loss: 0.8326 - acc: 0.7821
Epoch 7/25
20/20 [==============================] - 9s 434ms/step - loss: 0.8511 - acc: 0.8752
Epoch 8/25
20/20 [==============================] - 8s 416ms/step - loss: 0.7280 - acc: 0.9152
Epoch 9/25
20/20 [==============================] - 8s 421ms/step - loss: 1.3343 - acc: 0.8682
Epoch 10/25
20/20 [==============================] - 9s 440ms/step - loss: 1.1967 - acc: 0.8914
Epoch 11/25
20/20 [==============================

In [11]:
test_generator.reset()
predict = model.predict_generator(test_generator, steps = len(test_generator.filenames))
len(predict)

744

In [12]:
predict

array([[9.78495777e-01],
       [1.00000000e+00],
       [1.56836375e-08],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [9.97876525e-01],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [9.99999762e-01],
       [0.00000000e+00],
       [1.00000000e+00],
       [0.00000000e+00],
       [1.77501549e-03],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [0.00000000e+00],
       [1.00000000e+00],
       [9.99996305e-01],
       [1.00000000e+00],
       [9.97068822e-01],
       [9.98116612e-01],
       [4.99762450e-07],
       [6.62375195e-03],
       [0.00000000e+00],
       [9.99697566e-01],
       [1.00000000e+00],
       [0.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [8.99259925e-01],
       [1.00000000e+00],
       [1.00000000e+00],


In [13]:
sub_df = pd.read_csv('../content/drive/My Drive/sample_submission.csv')
sub_df.head()

,id,label
0,0,dirty
1,1,dirty
2,2,dirty
3,3,dirty
4,4,dirty


In [14]:
sub_df['label'] = predict
sub_df['label'] = sub_df['label'].apply(lambda x: 'dirty' if x > 0.5 else 'cleaned')
sub_df.head()

,id,label
0,0,dirty
1,1,dirty
2,2,cleaned
3,3,dirty
4,4,dirty


In [15]:
sub_df['label'].value_counts()

dirty      534
cleaned    210
Name: label, dtype: int64

In [0]:
sub_df.to_csv('sub.csv', index=False)